In [1]:
import backtesting
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas as pd
from bokeh.io import output_notebook
output_notebook()
backtesting.set_bokeh_output(notebook=True)

Loading BokehJS ...

In [2]:
tsla_data = pd.read_csv("data/TSLA.csv")
tsla_data["Date"] = pd.to_datetime(tsla_data["Date"])
tsla_data = tsla_data[(tsla_data["Date"]>= '01/01/2022') & (tsla_data["Date"]>'01/01/2023')]
tsla_data.set_index("Date", inplace = True)
# tsla_data = pd.Series(tsla_data["Adj Close"].values, index = tsla_data["Date"])

In [3]:
class MA_crossover_bt(Strategy):
    short_period = 10
    long_period = 50

    def init(self):
        # Inicializando as médias móveis: curta e longa
        self.short_ma = self.I(self.moving_average, self.data.Close, self.short_period)
        self.long_ma = self.I(self.moving_average, self.data.Close, self.long_period)

    def moving_average(self, data, size):
        """Função para calcular a média móvel exponencial dos dados"""
        return pd.Series(data).ewm(span=size, adjust=False).mean()

    def next(self):

        # Verifica a média móvel de curto prazo cruzou acima da média móvel de longo prazo
        if crossover(self.short_ma, self.long_ma):
            self.position.close() # Fecha a posição atual
            self.buy() # Realiza uma ordem de compra

        # Verifica se a média móvel de longo prazo cruzou acima da média móvel de curto prazo    
        elif crossover(self.long_ma, self.short_ma):
            self.position.close() # Fecha a posição atual
            self.sell() # Realiza uma ordem de venda

In [4]:
# Parâmetros da estratégia
short_period = 10
long_period = 50

# Executando o backtest
bt = Backtest(tsla_data, MA_crossover_bt, cash=10_000, commission=0)

# Executa o backtest
stats = bt.run()

# Exibe os resultados
print(stats)

Start                     2023-01-03 00:00:00
End                       2024-08-21 00:00:00
Duration                    596 days 00:00:00
Exposure Time [%]                   81.508516
Equity Final [$]                  5406.898295
Equity Peak [$]                  13635.439774
Return [%]                         -45.931017
Buy & Hold Return [%]              106.540248
Return (Ann.) [%]                  -31.410094
Volatility (Ann.) [%]               30.435927
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -60.358461
Avg. Drawdown [%]                  -19.550368
Max. Drawdown Duration      400 days 00:00:00
Avg. Drawdown Duration       95 days 00:00:00
# Trades                                   16
Win Rate [%]                            18.75
Best Trade [%]                      22.459725
Worst Trade [%]                    -16.534617
Avg. Trade [%]                    

In [5]:
stats.iloc[0]

Timestamp('2023-01-03 00:00:00')

In [6]:
bt.plot()


GridPlot(id='p1338', ...)

In [7]:
class MACD(Strategy):
    """ ADD stop loss"""
    short_period = 12
    long_period = 26   
    signal_period = 9 
    

    def init(self):
        # Calculate short and long moving averages
        self.short_ma = self.I(self.moving_average, self.data.Close, self.short_period)
        self.long_ma = self.I(self.moving_average, self.data.Close, self.long_period)
        
        # MACD line: difference between short and long moving averages
        self.macd_line = self.short_ma - self.long_ma
        
        # Signal line: EMA of the MACD line
        self.signal_line = self.I(self.moving_average, self.macd_line, self.signal_period)

    def moving_average(self, data, size):
        """Exponential moving average"""
        return pd.Series(data).ewm(span=size, adjust=False).mean()

    def next(self):
        # Buy when MACD line crosses above the signal line
        if crossover(self.macd_line, self.signal_line):
            if self.position.is_short:
                self.position.close()  # Close any short positions before buying
            self.buy()

        # Sell when MACD line crosses below the signal line
        elif crossover(self.signal_line, self.macd_line):
            if self.position.is_long:
                self.position.close()  # Close any long positions before selling
            self.sell()

In [8]:
# Executando o backtest
bt = Backtest(tsla_data, MACD, cash=10_000, commission=0)

# Executa o backtest
stats = bt.run()

# Exibe os resultados
print(stats)

Start                     2023-01-03 00:00:00
End                       2024-08-21 00:00:00
Duration                    596 days 00:00:00
Exposure Time [%]                   91.727494
Equity Final [$]                 24351.683074
Equity Peak [$]                  27672.734767
Return [%]                         143.516831
Buy & Hold Return [%]              106.540248
Return (Ann.) [%]                   72.582149
Volatility (Ann.) [%]               87.264191
Sharpe Ratio                         0.831752
Sortino Ratio                         2.44232
Calmar Ratio                         2.611953
Max. Drawdown [%]                   -27.78846
Avg. Drawdown [%]                   -6.623716
Max. Drawdown Duration      148 days 00:00:00
Avg. Drawdown Duration       26 days 00:00:00
# Trades                                   26
Win Rate [%]                        57.692308
Best Trade [%]                      40.018423
Worst Trade [%]                    -14.327409
Avg. Trade [%]                    

In [9]:
bt.plot()

GridPlot(id='p1754', ...)

# Boillinger Bands

In [10]:
import numpy as np

In [11]:
class BollingerBands(Strategy):
    period = 20  # Standard period for Bollinger Bands
    std_dev = 2  # Number of standard deviations for the bands
    stop_loss_pct = 0.05  # Stop loss at 5%

    def init(self):
        # Compute moving average (SMA)
        self.ma = self.I(self.moving_average, self.data.Close, self.period)

        # Compute rolling standard deviation (compatible with Backtesting.py)
        self.std = self.I(self.rolling_std, self.data.Close, self.period)

        # Compute upper and lower Bollinger Bands
        self.upper_band = self.I(lambda: self.ma + self.std * self.std_dev, name = "Upper Band")
        self.lower_band = self.I(lambda: self.ma - self.std * self.std_dev, name = "Lower Band")

    def moving_average(self, data, period):
        """Simple moving average (SMA)"""
        return pd.Series(data).ewm(span=period, adjust=False).mean()

    def rolling_std(self, data, period):
        """Rolling standard deviation"""
        std = pd.Series(data).rolling(window=period).std()
        return std.values

    def next(self):
        # If the price crosses below the lower band, we buy
        if crossover(self.lower_band, self.data.Close):
            if self.position.is_short:
                self.position.close()  # Close any short positions before buying
            self.buy()

        # If the price crosses above the upper band, we sell
        elif crossover(self.data.Close, self.upper_band):
            if self.position.is_long:
                self.position.close()  # Close any long positions before selling
            self.sell()

        # Implement stop loss
        if self.position:
            if self.position.is_long and hasattr(self.position, 'price'):
                if self.data.Close[-1] < self.position.price * (1 - self.stop_loss_pct):
                    self.position.close()
            elif self.position.is_short and hasattr(self.position, 'price'):
                if self.data.Close[-1] > self.position.price * (1 + self.stop_loss_pct):
                    self.position.close()

In [12]:
# Executando o backtest
bt = Backtest(tsla_data, BollingerBands, cash=10_000, commission=0)

# Executa o backtest
stats = bt.run()

# Exibe os resultados
print(stats)

Start                     2023-01-03 00:00:00
End                       2024-08-21 00:00:00
Duration                    596 days 00:00:00
Exposure Time [%]                   81.751825
Equity Final [$]                 25630.998381
Equity Peak [$]                  34989.437871
Return [%]                         156.309984
Buy & Hold Return [%]              106.540248
Return (Ann.) [%]                   78.086072
Volatility (Ann.) [%]                104.4819
Sharpe Ratio                         0.747365
Sortino Ratio                        2.439069
Calmar Ratio                         1.663827
Max. Drawdown [%]                  -46.931596
Avg. Drawdown [%]                   -6.534237
Max. Drawdown Duration      119 days 00:00:00
Avg. Drawdown Duration       17 days 00:00:00
# Trades                                    8
Win Rate [%]                             75.0
Best Trade [%]                      76.061886
Worst Trade [%]                    -19.355703
Avg. Trade [%]                    

In [14]:
# Plotagem do backtest
bt.plot()

GridPlot(id='p2609', ...)